In [15]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):   
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  logdir = os.path.join('logs', datetime.datetime.now().strftime('%y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  
  
  model.fit(
      X_train, 
      y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs = params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensorboard_callback],  
  )

  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax( y_test, axis=1)
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax( y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),                     
    Dropout(0.3),

    Dense(1024, activation='relu'),                     
    Dropout(0.3),

    Dense(num_classes, activation='softmax')
  ])


In [13]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 5s 18ms/step - loss: 2.9405 - accuracy: 0.2285 - val_loss: 1.5270 - val_accuracy: 0.5044
Epoch 2/5
272/272 [==============================] - 5s 17ms/step - loss: 1.2722 - accuracy: 0.5871 - val_loss: 0.4818 - val_accuracy: 0.8560
Epoch 3/5
272/272 [==============================] - 5s 17ms/step - loss: 0.6057 - accuracy: 0.8133 - val_loss: 0.1360 - val_accuracy: 0.9637
Epoch 4/5
272/272 [==============================] - 5s 17ms/step - loss: 0.3535 - accuracy: 0.8921 - val_loss: 0.0862 - val_accuracy: 0.9763
Epoch 5/5
272/272 [==============================] - 5s 17ms/step - loss: 0.2528 - accuracy: 0.9226 - val_loss: 0.0628 - val_accuracy: 0.9828


0.9591836734693877

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),                     
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),                     
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax')
  ])


In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  model.fit(
      X_train, 
      y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs = 5,
      verbose = 0,  
  )

  score  = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [29]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials(),
)

{'batch_size': 180.0, 'dropout_cnn_block_one': 0.3538262515042258, 'dropout_cnn_block_three': 0.34018462401900723, 'dropout_cnn_block_two': 0.39559500359530814, 'dropout_dense_block_one': 0.5068428013305013, 'dropout_dense_block_two': 0.5503776324356227}
accuracy=0.9478458166122437
{'batch_size': 100.0, 'dropout_cnn_block_one': 0.4909773364296268, 'dropout_cnn_block_three': 0.3233600543133675, 'dropout_cnn_block_two': 0.4646265837278438, 'dropout_dense_block_one': 0.6105703532307469, 'dropout_dense_block_two': 0.3591916537652406}
accuracy=0.8895691633224487
{'batch_size': 150.0, 'dropout_cnn_block_one': 0.4189320208146964, 'dropout_cnn_block_three': 0.4157661621722655, 'dropout_cnn_block_two': 0.44384983999612493, 'dropout_dense_block_one': 0.6938893176613581, 'dropout_dense_block_two': 0.30365938373022056}
accuracy=0.8895691633224487
{'batch_size': 100.0, 'dropout_cnn_block_one': 0.39301793210484454, 'dropout_cnn_block_three': 0.4748080119422176, 'dropout_cnn_block_two': 0.41097641000